conda environment to use on nightingale: /scratch/conda_envs/elmo-embeddings

In [1]:
%load_ext autoreload

In [2]:
%autoreload

from allennlp.commands.elmo import ElmoEmbedder
import os
from sys import path
path.append('..')
import utils
import h5py
import numpy as np
data_path = '/data/medg/misc/semeval_2010'
def res(path): return os.path.join(data_path, path)

ModuleNotFoundError: No module named 'allennlp'

In [84]:
elmo = ElmoEmbedder()

In [4]:
tokens = ["I", "ate", "an", "apple", "for", "breakfast"]

In [11]:
vectors = elmo.embed_sentence(tokens)
assert(len(vectors) == 3) # one for each layer in the ELMo output
assert(len(vectors[0]) == len(tokens)) # the vector elements correspond with the input tokens

In [10]:
# the dimensionality of this vector is layer in ELMO LSTM X num_of_words_in_sentence X the size of the ELMO embedding
vectors

array([[[ 6.9227183e-01, -3.2613143e-01,  2.2827488e-01, ...,
          1.7574835e-01,  2.6598686e-01, -1.0131977e-01],
        [-6.6311550e-01,  2.9512233e-01,  6.5207249e-01, ...,
          6.5627491e-01,  4.2394802e-01,  1.2068851e+00],
        [ 7.9657093e-02,  1.9919696e-01, -6.9490165e-02, ...,
          2.1669537e-02,  1.2296094e-01,  4.1095167e-03],
        [ 1.4436811e-01,  6.7775488e-02,  3.7736315e-01, ...,
          4.1031554e-01,  2.9029346e-01, -6.1045110e-02],
        [-2.4150278e-01,  5.4133482e-02, -3.1142172e-01, ...,
          4.0102762e-01, -2.6212478e-01, -4.2983246e-01],
        [-2.3136714e-01,  2.7052855e-01,  4.4680029e-01, ...,
          2.0299840e-01, -2.5246492e-01,  4.1547477e-02]],

       [[-1.1051465e+00, -4.0921751e-01, -4.3645084e-01, ...,
         -5.5361629e-01, -2.2313195e-01,  3.2954216e-02],
        [ 2.3847297e-01, -2.4085957e-01,  2.2867355e-02, ...,
          3.5999560e-01, -1.5432239e-02,  9.6235260e-02],
        [ 6.7190811e-02, -1.4298746e-0

In order to incorporate ELMO into my model (well the very first thing is just to use the first layer) and then beyond that use the other layers. But in order to map a sentence to the ELMO representation (which is the larger goal), I should re-generate my pickle files to include the embedding of the sentence with it so I can finally use a linearly weighted combinations of the embeddings from the different layers of the LSTM. One important question is though, that they claim that the very first layer includes context insensitive embeddings. Does this mean that in two sentences, if I have the word "claims," they would both have the same vector? Let's check

In [13]:
new_sentence = ["The", "apple", "that", "I", "ate", "was", "not", "delicious"]
vector2 = elmo.embed_sentence(new_sentence)
assert(len(vector2) == 3)
assert(len(vector2[0] == len(new_sentence)))

In [15]:
vector2[0][2]

array([-0.10939915, -0.1721538 ,  0.11998197, ...,  0.14708203,
        0.6417191 , -0.5814706 ], dtype=float32)

In [16]:
vectors[0][3]

array([ 0.14436811,  0.06777549,  0.37736315, ...,  0.41031554,
        0.29029346, -0.06104511], dtype=float32)

Nope so the embeddings are different, so this is not the best way to generate individual word embeddings via elmo

# Simplest possible way of generating embedding file similar to Glove/Senna etc

Read the train and test file of Semeval data, get the tokenized version of the sentence and then feed in individual word to generate an embedding for

In [22]:
res('train.txt')

'/data/medg/misc/semeval_2010/train.txt'

In [64]:
def get_duplicate_words_from_file(filename):
    words = []
    with open(res(filename)) as f:
        for line in f:
            line = line.strip().lower().split()
            sentence = line[5:]
            words.extend(sentence)
    return words

In [65]:
dup_words = get_duplicate_words_from_file('train.txt') + get_duplicate_words_from_file('test.txt')

In [66]:
words = list(set(dup_words))

In [67]:
vectorlist = []

In [68]:
# consider doing some pre processing of the data where you either turn it into a number or some range of numbers

In [69]:
for word in words:
    vectors = elmo.embed_sentence([word])
    assert(len(vectors) == 3) # one for each layer in the ELMo output
    assert(len(vectors[0]) == 1) # the vector elements correspond with the input tokens
    # grab the first layer and then write that into a file
    vector = vectors[0][0]
    list_a = [word]
    list_a.extend(vector)
    vectorlist.append(list_a)

In [48]:
# word = words[0]
# vectors = elmo.embed_sentence([word])
# assert(len(vectors) == 3) # one for each layer in the ELMo output
# assert(len(vectors[0]) == 1) # the vector elements correspond with the input tokens
# # grab the first layer and then write that into a file
# vector = vectors[0][0]
# list_a = [word]
# list_a.extend(vector)
# vectorlist.append(list_a)

In [71]:
vectorlist

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [81]:
with open(res('elmo/elmo.layer1.txt'), 'w') as f:
    for vector in vectorlist:
        vectorstring = ""
        for val in vector:
            vectorstring = vectorstring + " " + str(val)
        vectorstring = vectorstring.strip()
        f.write(vectorstring + "\n")
    f.close()

## Next, work on getting all the layers of the elmo embeddings

In [172]:
def get_sentences(filename):
    data = open(res(filename))
    data = utils.split_data_cut_sentence(data, 50)
    sentences = data[0]
    return sentences

In [173]:
train_sentences = get_sentences('train.txt')

In [175]:
test_sentences = get_sentences('test.txt')

In [177]:
len(train_sentences)

8000

In [176]:
len(test_sentences)

2717

In [100]:
def write_sentences_to_txt(sentences, filename):
    with open(res('elmo/input-sentences/'+filename), 'w') as f:
        for sentence in sentences:
            sentence_string = " ".join(sentence)
            f.write(sentence_string + "\n")

In [101]:
write_sentences_to_txt(train_sentences, 'train-sentences.txt')

In [102]:
write_sentences_to_txt(test_sentences, 'test-sentences.txt')

Now, follow the instructions on https://github.com/allenai/allennlp/blob/master/tutorials/how_to/elmo.md to generate the h5py file 

Nope, that does not work

In [ ]:
with open(res('elmo/input-sentences/train-sentences.txt')) as train_file:
    elmo.embed_file(input_file=train_file, 
                    output_file_path = res('elmo/train-elmo-full.hdf5'))


0it [00:00, ?it/s]
1it [00:37, 37.34s/it]
65it [01:31, 26.39s/it]
129it [02:16, 18.69s/it]
193it [02:59, 13.28s/it]
257it [03:39,  9.49s/it]
321it [04:19,  6.83s/it]
385it [04:57,  4.96s/it]
449it [05:38,  3.66s/it]
513it [06:13,  2.73s/it]
577it [06:58,  2.12s/it]
641it [07:37,  1.67s/it]
705it [08:12,  1.33s/it]
769it [08:43,  1.08s/it]
833it [09:25,  1.05it/s]
897it [10:03,  1.19it/s]
961it [10:35,  1.35it/s]
1025it [11:18,  1.39it/s]
1089it [11:46,  1.57it/s]
1153it [12:18,  1.69it/s]
1217it [12:52,  1.74it/s]
1281it [13:33,  1.68it/s]
1345it [14:09,  1.70it/s]
1409it [14:47,  1.70it/s]
1473it [15:43,  1.48it/s]
1537it [16:17,  1.59it/s]
1601it [16:53,  1.63it/s]
1665it [17:31,  1.66it/s]
1729it [18:05,  1.71it/s]
1793it [18:53,  1.57it/s]
1857it [19:26,  1.68it/s]
1921it [20:00,  1.73it/s]
1985it [20:38,  1.72it/s]
2049it [21:27,  1.56it/s]
2113it [22:05,  1.60it/s]
2177it [22:44,  1.62it/s]
2241it [23:20,  1.66it/s]
2305it [24:02,  1.62it/s]
2369it [24:38,  1.65it/s]
2433it [25:

In [112]:
with open(res('elmo/input-sentences/test-sentences.txt')) as test_file:
    elmo.embed_file(input_file=test_file, 
                    output_file_path = res('elmo/test-elmo-full.hdf5'))


0it [00:00, ?it/s]
1it [00:04,  4.55s/it]
65it [00:09,  3.20s/it]
129it [00:13,  2.26s/it]
193it [00:16,  1.60s/it]
257it [00:19,  1.13s/it]
321it [00:22,  1.24it/s]
385it [00:25,  1.73it/s]
449it [00:29,  2.37it/s]
513it [00:32,  3.19it/s]
518it [00:33,  4.37it/s]
577it [00:37,  5.41it/s]
641it [00:42,  6.65it/s]
705it [00:46,  7.95it/s]
769it [00:50,  9.47it/s]
833it [00:54, 10.88it/s]
897it [00:57, 12.76it/s]
961it [01:00, 14.37it/s]
1025it [01:04, 15.22it/s]
1089it [01:08, 15.29it/s]
1153it [01:11, 16.85it/s]
1161it [01:11, 20.90it/s]
1217it [01:15, 17.65it/s]
1281it [01:20, 16.47it/s]
1284it [01:20, 16.55it/s]
1345it [01:23, 17.60it/s]
1409it [01:27, 16.69it/s]
1473it [01:31, 16.33it/s]
1537it [01:34, 17.41it/s]
1601it [01:38, 17.14it/s]
1665it [01:42, 17.66it/s]
1729it [01:46, 16.81it/s]
1793it [01:50, 16.13it/s]
1857it [01:54, 16.46it/s]
1921it [01:57, 17.05it/s]
1926it [01:57, 19.71it/s]
1985it [02:00, 19.71it/s]
2049it [02:05, 18.29it/s]
2113it [02:08, 18.95it/s]
2177it [02:1

## Read the h5py file and write it as a np array on disk

In [251]:
def get_elmo_embeddings(filename):
    h5py_file = h5py.File(filename, 'r')
    elmo_embeddings = []
    for i in range(0, len(h5py_file) - 1):
        embedding = h5py_file.get(str(i))
        elmo_embeddings.append(np.array(embedding))
    return elmo_embeddings

In [253]:
elmo_embedding_train = get_elmo_embeddings(res('elmo/train-elmo-full.hdf5'))

In [255]:
elmo_embedding_test = get_elmo_embeddings(res('elmo/test-elmo-full.hdf5'))

In [257]:
np.save(res('elmo/train-elmo-full.npy'), elmo_embedding_train)

ValueError: could not broadcast input array from shape (3,17,1024) into shape (3)

In [191]:
h5py_file = h5py.File("/data/medg/misc/semeval_2010/elmo/train-elmo-full.hdf5", 'r')

In [192]:
embedding = h5py_file.get("0")

In [193]:
embedding

<HDF5 dataset "0": shape (3, 17, 1024), type "<f4">

In [194]:
np.array(embedding).shape

(3, 17, 1024)

In [195]:
assert(len(embedding) == 3)

In [196]:
len(h5py_file)

8001

8000 + (1 is extra) for all the sentences in the training set

In [197]:
# dimensions is layers, words_in_sentences, dimensions_per_layer
elmo_embed_train = []
for i in range(0, len(h5py_file) - 1):
    embedding = h5py_file.get(str(i))
    elmo_embed_train.append(np.array(embedding))

In [198]:
len(elmo_embed_train)

8000

In [134]:
elmo_embed_train[0].shape
first_sentence = elmo_embed_train[0]

In [199]:
first_sentence.shape

(3, 17, 1024)

In [200]:
max_len = 0
# because of the new line character, elmo generated embeddings for the blank line at the end. 
for i in range(0, len(elmo_embed_train)):
    sentence = elmo_embed_train[i]
    if sentence.shape[1] > max_len:
        max_len = sentence.shape[1]

## create a new embedding with the padding

In [202]:
new_elmo_embed_train=[]
for i in range(0, len(elmo_embed_train)):
    sentence = elmo_embed_train[i]
    num_of_words_to_pad = max_len - sentence.shape[1] # gives the length of the sentence
    array_to_pad = np.zeros(shape=(sentence.shape[0], num_of_words_to_pad, sentence.shape[2]), dtype='float32')
    appended_array = np.append(sentence, array_to_pad, axis=1) # appending along the sentence axis
    new_elmo_embed_train.append(appended_array)

try to pad the elmo_embed_train with 0's in the second dimension acc to max sentence length 18

In [141]:
a = np.array([1,2,3], dtype='float32')

In [145]:
np.append(a, 0)

array([1., 2., 3., 0.])

In [148]:
toappend = np.zeros(shape=(3,1,1024), dtype='float32')

In [150]:
appended = np.append(first_sentence, toappend, axis=1)

In [151]:
appended.shape

(3, 18, 1024)

Let's make sure to see with a simple example

In [153]:
a = np.zeros(shape=(3,4,2), dtype='float32')

In [154]:
b = np.ones(shape=(3,1,2), dtype='float32')

In [157]:
np.append(a, b, axis=1)

array([[[0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [1., 1.]],

       [[0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [1., 1.]],

       [[0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [1., 1.]]], dtype=float32)

In [152]:
elmo_embed_train[0].shape

(3, 17, 1024)

next will need to see how to append to the original train array

In [239]:
sentences = np.zeros((3,5), dtype=int)

In [240]:
e1 = np.ones((3,1), dtype=int)

In [241]:
batch = sentences, e1

In [231]:
batch

(array([[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]]), array([[1],
        [1],
        [1]]))

In [242]:
batch = list(zip(*batch))

In [243]:
batch = (x for x in zip(*batch))

In [244]:
batch

<generator object <genexpr> at 0x7f85a6a72360>

In [245]:
(sentences, e1) = batch

In [249]:
sentences

(array([0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0]))

In [250]:
np.vstack(sentences)

array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]])

In [247]:
e1

(array([1]), array([1]), array([1]))